# Tool Result Cache Middleware with LangChain Agents

This notebook demonstrates how to use `ToolResultCacheMiddleware` with LangChain agents using the standard `create_agent` pattern. The middleware caches tool execution results, reducing latency and costs for expensive operations.

## Key Features

- **Tool metadata support**: Use LangChain's native `tool.metadata = {"cacheable": True/False}`
- **Config-based fallback**: Configure default caching behavior via `cacheable_tools` / `excluded_tools`
- **Semantic matching**: Match similar tool calls by arguments
- **TTL support**: Automatic cache expiration

## Cacheability Rules

1. **Tool metadata takes precedence**: If a tool has `metadata={"cacheable": False}`, it won't be cached regardless of config
2. **Config as fallback**: When no metadata is set, `cacheable_tools` and `excluded_tools` config options apply
3. **Safe defaults**: Tools with temporal/non-deterministic results should set `metadata={"cacheable": False}`

## Use Cases

- ✅ Cache: Deterministic calculations, static lookups, expensive but stable API calls
- ❌ Don't cache: Random number generators, real-time data (stock prices, weather), database queries on changing data

## Prerequisites

- Redis 8.0+ or Redis Stack (with RedisJSON and RediSearch)
- OpenAI API key

## Setup

Install required packages and set API keys.

In [1]:
%%capture --no-stderr
%pip install -U langgraph-checkpoint-redis langchain langchain-openai sentence-transformers

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:  ········


## Using ToolResultCacheMiddleware with create_agent

The `ToolResultCacheMiddleware` inherits from LangChain's `AgentMiddleware`, so it can be passed directly to `create_agent`.

In [ ]:
import ast
import operator as op
import random
import time

from langchain_core.tools import tool

# Safe math evaluator - no arbitrary code execution
SAFE_OPS = {
    ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
    ast.Div: op.truediv, ast.Pow: op.pow, ast.USub: op.neg,
}

def _eval_node(node):
    if isinstance(node, ast.Constant):
        return node.value
    elif isinstance(node, ast.BinOp) and type(node.op) in SAFE_OPS:
        return SAFE_OPS[type(node.op)](_eval_node(node.left), _eval_node(node.right))
    elif isinstance(node, ast.UnaryOp) and type(node.op) in SAFE_OPS:
        return SAFE_OPS[type(node.op)](_eval_node(node.operand))
    raise ValueError("Unsupported expression")

def safe_eval(expr: str) -> float:
    return _eval_node(ast.parse(expr, mode='eval').body)

# Track actual tool executions
tool_execution_count = {"search": 0, "calculate": 0, "random_number": 0, "get_stock_price": 0}


@tool
def search(query: str) -> str:
    """Search the web for information. Results are relatively stable."""
    tool_execution_count["search"] += 1
    time.sleep(1.0)  # Simulate API call
    return f"Search results for '{query}': Found 10 relevant articles about {query}."


# Mark as cacheable via metadata (deterministic operation)
search.metadata = {"cacheable": True}


@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression. Always deterministic!"""
    tool_execution_count["calculate"] += 1
    time.sleep(0.5)  # Simulate computation
    try:
        result = safe_eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error: {str(e)}"


# Mark as cacheable via metadata (deterministic operation)
calculate.metadata = {"cacheable": True}


@tool
def random_number(max_value: int) -> str:
    """Generate a random number. Non-deterministic - should NOT be cached!"""
    tool_execution_count["random_number"] += 1
    return f"Random number: {random.randint(1, max_value)}"


# Mark as NOT cacheable via metadata (non-deterministic)
random_number.metadata = {"cacheable": False}


@tool
def get_stock_price(symbol: str) -> str:
    """Get current stock price. Temporal data - should NOT be cached!"""
    tool_execution_count["get_stock_price"] += 1
    # Simulate real-time price
    price = 150.00 + random.uniform(-5, 5)
    return f"{symbol}: ${price:.2f}"


# Mark as NOT cacheable via metadata (temporal data)
get_stock_price.metadata = {"cacheable": False}


tools = [search, calculate, random_number, get_stock_price]

print("Tools defined with cacheability metadata:")
for t in tools:
    cacheable = t.metadata.get("cacheable", "not set") if t.metadata else "not set"
    print(f"  - {t.name}: cacheable={cacheable}")

In [4]:
from langchain.agents import create_agent

from langgraph.middleware.redis import ToolCacheConfig, ToolResultCacheMiddleware

REDIS_URL = os.environ.get("REDIS_URL", "redis://redis:6379")

# Create the tool cache middleware
# Note: Tool metadata takes precedence over these config options
tool_cache = ToolResultCacheMiddleware(
    ToolCacheConfig(
        redis_url=REDIS_URL,
        name="demo_tool_cache",
        # These are fallbacks when tool.metadata["cacheable"] is not set:
        # cacheable_tools=["search", "calculate"],
        # excluded_tools=["random_number"],
        distance_threshold=0.1,  # Strict matching for tools
        ttl_seconds=1800,  # 30 minutes
    )
)

print("ToolResultCacheMiddleware created!")
print("\nCacheability is determined by tool.metadata['cacheable']:")
print("  - search: cacheable=True (stable results)")
print("  - calculate: cacheable=True (deterministic)")
print("  - random_number: cacheable=False (non-deterministic)")
print("  - get_stock_price: cacheable=False (temporal data)")

ToolResultCacheMiddleware created!

Cacheability is determined by tool.metadata['cacheable']:
  - search: cacheable=True (stable results)
  - calculate: cacheable=True (deterministic)
  - random_number: cacheable=False (non-deterministic)
  - get_stock_price: cacheable=False (temporal data)


In [5]:
# Create the agent with tool cache middleware
agent = create_agent(
    model="gpt-4o-mini",
    tools=tools,
    middleware=[tool_cache],  # Pass middleware here!
)

print("Agent created with ToolResultCacheMiddleware!")

Agent created with ToolResultCacheMiddleware!


## Demonstrating Tool Cache Behavior

Let's make queries that trigger tool calls and observe how caching works.

**Important**: We use `await agent.ainvoke()` because the middleware is async-first.

In [6]:
from langchain_core.messages import HumanMessage

# Reset counters
tool_execution_count = {"search": 0, "calculate": 0, "random_number": 0, "get_stock_price": 0}

# First search query - tool will execute
print("Query 1: 'Search for Python tutorials'")
print("=" * 50)

start = time.time()
result1 = await agent.ainvoke({"messages": [HumanMessage(content="Search for Python tutorials")]})
elapsed1 = time.time() - start

print(f"Response: {result1['messages'][-1].content[:150]}...")
print(f"Time: {elapsed1:.2f}s")
print(f"Tool executions: {tool_execution_count}")

Query 1: 'Search for Python tutorials'
Response: Here are some resources for Python tutorials:

1. **Official Python Documentation**: A great place to start learning Python with official guides and t...
Time: 8.51s
Tool executions: {'search': 1, 'calculate': 0, 'random_number': 0, 'get_stock_price': 0}


In [7]:
# Similar search query - should hit cache
print("\nQuery 2: 'Find Python tutorials online'")
print("=" * 50)

start = time.time()
result2 = await agent.ainvoke({"messages": [HumanMessage(content="Find Python tutorials online")]})
elapsed2 = time.time() - start

print(f"Response: {result2['messages'][-1].content[:150]}...")
print(f"Time: {elapsed2:.2f}s (expected: faster due to cache!)")
print(f"Tool executions: {tool_execution_count}")


Query 2: 'Find Python tutorials online'
Response: Here are some online Python tutorials you can explore:

1. **W3Schools** - Offers a comprehensive Python tutorial covering basic to advanced topics.
 ...
Time: 13.14s (expected: faster due to cache!)
Tool executions: {'search': 2, 'calculate': 0, 'random_number': 0, 'get_stock_price': 0}


In [8]:
# Calculate query - will be cached
print("\nQuery 3: 'Calculate 25 * 4 + 100'")
print("=" * 50)

start = time.time()
result3 = await agent.ainvoke({"messages": [HumanMessage(content="Calculate 25 * 4 + 100")]})
elapsed3 = time.time() - start

print(f"Response: {result3['messages'][-1].content}")
print(f"Time: {elapsed3:.2f}s")
print(f"Tool executions: {tool_execution_count}")


Query 3: 'Calculate 25 * 4 + 100'
Response: The result of the calculation \( 25 \times 4 + 100 \) is 200.
Time: 2.20s
Tool executions: {'search': 2, 'calculate': 1, 'random_number': 0, 'get_stock_price': 0}


In [9]:
# Non-cacheable tools - should execute every time
print("\nQuery 4: Testing non-cacheable tools")
print("=" * 50)

# Random number - marked cacheable=False
print("\nRandom numbers (cacheable=False):")
for i in range(3):
    result = await agent.ainvoke({"messages": [HumanMessage(content="Generate a random number up to 100")]})
    print(f"  Attempt {i + 1}: {result['messages'][-1].content}")

# Stock price - marked cacheable=False (temporal)
print("\nStock prices (cacheable=False - temporal data):")
for i in range(2):
    result = await agent.ainvoke({"messages": [HumanMessage(content="What is the stock price of AAPL?")]})
    print(f"  Attempt {i + 1}: {result['messages'][-1].content}")

print(f"\nTool executions: {tool_execution_count}")
print("\nNote: Tools with metadata={'cacheable': False} are never cached!")


Query 4: Testing non-cacheable tools

Random numbers (cacheable=False):
  Attempt 1: The random number generated is 62.
  Attempt 2: The random number generated is 56.
  Attempt 3: The random number generated is 20.

Stock prices (cacheable=False - temporal data):
  Attempt 1: The stock price of AAPL (Apple Inc.) is $149.16.
  Attempt 2: The current stock price of AAPL (Apple Inc.) is $149.91.

Tool executions: {'search': 2, 'calculate': 1, 'random_number': 3, 'get_stock_price': 2}

Note: Tools with metadata={'cacheable': False} are never cached!


In [10]:
# Summary
print("\n" + "=" * 50)
print("SUMMARY")
print("=" * 50)
print(f"Total tool executions: {tool_execution_count}")
print("\nCacheable tools (metadata={'cacheable': True}):")
print(f"  - search: {tool_execution_count['search']} executions")
print(f"  - calculate: {tool_execution_count['calculate']} executions")
print("\nNon-cacheable tools (metadata={'cacheable': False}):")
print(f"  - random_number: {tool_execution_count['random_number']} executions (never cached)")
print(f"  - get_stock_price: {tool_execution_count['get_stock_price']} executions (never cached)")
print("\nKey takeaway: Use tool.metadata={'cacheable': True/False} to control caching!")


SUMMARY
Total tool executions: {'search': 2, 'calculate': 1, 'random_number': 3, 'get_stock_price': 2}

Cacheable tools (metadata={'cacheable': True}):
  - search: 2 executions
  - calculate: 1 executions

Non-cacheable tools (metadata={'cacheable': False}):
  - random_number: 3 executions (never cached)
  - get_stock_price: 2 executions (never cached)

Key takeaway: Use tool.metadata={'cacheable': True/False} to control caching!


## Cleanup

In [11]:
# Close the middleware to release Redis connection
await tool_cache.aclose()
print("Middleware closed.")
print("Demo complete!")

Middleware closed.
Demo complete!
